# Redes Neurais
## 1ª Lista de Exercícios - Classificação de padrões


Aluno:

Base de dados: 

In [1]:
import numpy as np 
import pandas as pd 
import json
import matplotlib.pyplot as plt 
import seaborn as sns 
import os

import torch 
import torch.nn as nn
from torch.utils.data import WeightedRandomSampler, Dataset, DataLoader 

from tqdm.notebook import tqdm

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

#Descomentar caso queira utilizar no Google Colab
#from google.colab import drive 
#drive.mount('/content/drive')

In [2]:
class CustomDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)



In [3]:
def piecewise_norm(vec,val,n):
    '''
    Função para normalização por partes. 
    Argumentos de entrada:
    - vec: vetor com valores de um atributo. 
    - val: valor (na escala original) que deve ocorrer a quebra entre retas.
    - n: valor (normalizado) que deve ocorrer a quebra entre retas.
    '''
    
    assert val > min(vec)
    
    return np.where(vec < val, n*(vec - min(vec))/(val - min(vec)), (1-n)*(vec - val)/(max(vec) - val) + n)

In [4]:
def transform_data(dataset,dataset_columns,normalize=True):
    '''
    Função para transformação de dados para a 1ª Lista de exercícios.
    A transformação possui duas etapas: codificação de variáveis categóricas e normalização de atributos numéricos.
    Parâmetros de entrada:
    - dataset: base de dados a ser ajeitado.
    - dataset_columns: dicionário contendo o nome da coluna e "classe", podendo ser 'numeric', 'categorical' ou 'target'
    - normalize: normalização de atributos utilizando min-max. Default: True.

    A função retorna um dataframe com os atributos categóricos codificados e normalizados (caso normalize = True).
    '''

    new_df = pd.DataFrame()

    #Varre todas as colunas do dataframe.
    for name in dataset.columns:

        #Avalia se a coluna está dentro da lista de colunas com descritores.
        if name in dataset_columns.keys():
            
            #Tratamento de dados utilizando o atributo categórico. No caso, é utilizada a codificação 1 of N.
            if dataset_columns[name] == 'categorical':
                raw_data = dataset[name].values
                d_encoder = LabelEncoder()
                d_encoder.fit(raw_data)
                d_encoded = d_encoder.transform(raw_data)
                #dummy_y = to_categorical(d_encoded)
                dim = len(d_encoder.classes_)
                dummy_y = np.eye(dim)[d_encoded]
                
                for (j,k) in enumerate(d_encoder.classes_):
                    new_df[f'{name}_{k}'] = dummy_y[:,j].astype('int')
            
            
            #Caso numérico, utilizando normalização min-max
            elif dataset_columns[name] == 'numeric':
                raw_data = dataset[name].values
                if normalize:
                    new_df[name] = (raw_data - min(raw_data))/(max(raw_data) - min(raw_data))
                else:
                    new_df[name] = raw_data
                
            #Caso 'original', sem mudança nos valores.
            elif dataset_columns[name] == 'original':
                new_df[name] = dataset[name].values
            
            #Caso target, sem mudança nos valores.
            elif dataset_columns[name] == 'target':
                new_df[name] = dataset[name].values
                
    return new_df

In [5]:
class NeuralNetwork(torch.nn.Module):
  '''
  Objeto criado para facilitar o desenvolvimento dos scripts das aulas práticas.
  Opção alternativa à criação externa fdo modelo. Basicamente serve como um 
  'agregador'  de trechos comuns para a criação do modelo. Por exemplo, ao invés
  de gastar n+1 linhas para a criação de um modelo com n camadas, podemos criar 
  o mesmo modelo com apenas uma linha.
  
  Parâmetros de entrada: 
  - hidden_neurons: Lista com a quantidade de neurônios na camada escondida. 
  - hidden_activation: Função de ativação para cada camada escondida. Aceita 
  como parâmetro string ou lista. Caso o parâmetro receba string, então a mesma
  função de ativação é utilizada para todas as camadas escondidas. Caso seja uma 
  lista, cada camada terá sua propria função de ativação definida pela lista.
  - output_activation: Função de ativação para a camada de saída.
  - lr: Taxa de aprendizado
  - n_input: Tamanho do vetor de entrada.
  - n_output: Saída do modelo.
  '''
  def __init__(self,hidden_neurons = 4, hidden_activation = 'relu', output_activation='softmax', lr = 0.05, n_input = 1, n_output = 1):
    # create model
    super(NeuralNetwork, self).__init__()

    self.activations = {'relu': nn.ReLU(), 'sigmoid':nn.Sigmoid(), 'softmax':nn.Softmax()}

    self.fc1 = nn.Linear(n_input, hidden_neurons)
    self.ha = self.activations[hidden_activation]
    self.fc2 = nn.Linear(hidden_neurons, n_output)
    #self.out = self.activations[output_activation]

  def forward(self,x):
    h = self.fc1(x)
    h1 = self.ha(h) 
    y = self.fc2(h1) 
    #y = self.out(h2)
    return y



  


In [115]:
def accuracy(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc


def train(model, train_loader, val_loader, epochs, device, lr, binary=True):

  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)

  history = {'acc_train' : [], 'loss_train': [], 'acc_val': [], 'loss_val': []}

  for e in tqdm(range(1, epochs+1)):
    
    y_hat = np.array([])

    train_epoch_loss = 0
    train_epoch_acc = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X, y = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X)
        
        loss = criterion(y_pred, y)
        acc = accuracy(y_pred, y)
        
        loss.backward()
        optimizer.step()
        
        train_epoch_loss += loss.item()
        train_epoch_acc += acc.item()
        y_p = torch.argmax(y_pred, dim=1)
        y_hat = np.concatenate((y_hat, y_p))
    
    _, val_loss, val_acc = evaluate(model, val_loader, criterion, device, binary=binary)

    history['acc_train'].append(train_epoch_acc)
    history['loss_train'].append(train_epoch_loss)
    history['acc_val'].append(val_acc)
    history['loss_val'].append(val_loss)
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.3f} | Val Loss: {val_loss/len(val_loader):.4f} | Train Acc: {train_epoch_acc/len(train_loader):.4f}| Val Acc: {val_acc/len(val_loader):.4f}')
  return history, y_hat


  

def evaluate(model, loader, criterion, device, binary=True):

  with torch.no_grad():
      val_epoch_loss = 0
      val_epoch_acc = 0
      
      model.eval()
      for X_batch, y_batch in loader:
          X, y = X_batch.to(device), y_batch.to(device)
          
          y_pred = model(X)
                      
          loss = criterion(y_pred, y)
          acc = accuracy(y_pred, y)
          
          val_epoch_loss += loss.item()
          val_epoch_acc += acc.item()
  return y_pred, val_epoch_loss, val_epoch_acc


In [116]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Preparação dos dados

Não esqueça de verificar o path para a pasta. Mude também o filename, de acordo com a base de dados.

In [117]:
#Mudar o rootname para o diretório atual do Notebook.
#root_name = '/content/drive/MyDrive/Monitoria_RN/pratica1 - classificacao/'
root_name = os.getcwd()
dataset = 'heart_disease'
filename = f'datasets\\{dataset}'

In [118]:
description_path = os.path.join(root_name,filename,'data_info.json')
with open(description_path,'r') as f:
    dataset_columns = json.load(f)

In [119]:
dataset_path = os.path.join(root_name, filename,f'class_{dataset}.csv')
dataset = pd.read_csv(dataset_path)

In [120]:
dataset

,Age,Sex,CP,Trestbps,Chol,Fbs,Restecg,Thalach,Exang,Oldpeak,Slope,Ca,Thal,Num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


## Parte I - Compreensão do problema e análise de variáveis



### 1)	Observe a base de dados do problema. Existem variáveis que podem ser eliminadas do dataset? Justifique.

#### Rascunho

#### Resposta

### 2)	Implemente técnicas de visualização de dados e seleção de variáveis para extrair características importantes sobre a base de dados. Explique a motivação destas técnicas e o que é possível inferir dos resultados obtidos.

#### Rascunho

#### Resposta

In [121]:
#sns.pairplot(dataset, hue="target", diag_kind="hist")

## 2.	Treinamento do modelo de Rede Neural

### 1)	Com as configurações do modelo MLP previamente definidas no script, faça o treinamento da Rede Neural sem normalizar os atributos numéricos. Comente o resultado obtido, baseado nas métricas de avaliação disponíveis (acurácia, precision, recall, F1-Score, Matriz de confusão, etc.)

#### Rascunho

In [122]:
new_df = transform_data(dataset,dataset_columns,normalize=True)
#new_df.head()

In [123]:
X = new_df.values[:,:-1].astype('float')
target = new_df.values[:,-1]

encoder = LabelEncoder()
y = encoder.fit_transform(target)

n_input = X.shape[1]

if np.unique(y).shape[0] <= 2:
  print('Binary classification')
  n_output = 1

else:
  print('Multiclass classification')
  n_output = len(encoder.classes_)
  #y = np.eye(n_output)[y]

Multiclass classification


In [124]:
batch_size = 32 
epochs = 50

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [126]:
train_dataset = CustomDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_dataset = CustomDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
test_dataset = CustomDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

In [127]:
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

In [128]:
model = NeuralNetwork(n_input = n_input, n_output=n_output,output_activation='sigmoid')

In [129]:
lr = 0.01
epochs = 150

history, y_hat = train(model, train_loader, val_loader, epochs=epochs, lr=lr, device=device, binary=False)

Epoch 001: | Train Loss: 1.503 | Val Loss: 1.4691 | Train Acc: 34.4286| Val Acc: 44.8980
Epoch 002: | Train Loss: 1.343 | Val Loss: 1.4105 | Train Acc: 63.8571| Val Acc: 44.8980
Epoch 003: | Train Loss: 1.191 | Val Loss: 1.3729 | Train Acc: 64.2857| Val Acc: 44.8980
Epoch 004: | Train Loss: 1.081 | Val Loss: 1.3570 | Train Acc: 64.2857| Val Acc: 44.8980
Epoch 005: | Train Loss: 1.024 | Val Loss: 1.3279 | Train Acc: 64.2857| Val Acc: 44.8980
Epoch 006: | Train Loss: 0.985 | Val Loss: 1.2696 | Train Acc: 64.2857| Val Acc: 44.8980
Epoch 007: | Train Loss: 0.944 | Val Loss: 1.2102 | Train Acc: 64.2857| Val Acc: 44.8980
Epoch 008: | Train Loss: 0.906 | Val Loss: 1.1733 | Train Acc: 64.2857| Val Acc: 44.8980
Epoch 009: | Train Loss: 0.879 | Val Loss: 1.1519 | Train Acc: 64.2857| Val Acc: 44.8980
Epoch 010: | Train Loss: 0.860 | Val Loss: 1.1375 | Train Acc: 64.2857| Val Acc: 44.8980
Epoch 011: | Train Loss: 0.846 | Val Loss: 1.1255 | Train Acc: 64.2857| Val Acc: 44.8980
Epoch 012: | Train Lo

#### Resposta

### 2)	Agora normalize os dados de entrada e treine novamente o modelo MLP. Avalie os resultados obtidos e comente o efeito da normalização no treinamento da Rede Neural.

#### Rascunho

In [22]:
new_df = transform_data(dataset,dataset_columns,normalize=True)
new_df.head()

,Age,Sex_0.0,Sex_1.0,CP_1.0,CP_2.0,CP_3.0,CP_4.0,Trestbps,Chol,Fbs_0.0,...,Ca_0.0,Ca_1.0,Ca_2.0,Ca_3.0,Ca_?,Thal_3.0,Thal_6.0,Thal_7.0,Thal_?,Num
0,0.708333,0,1,1,0,0,0,0.481132,0.244292,0,...,1,0,0,0,0,0,1,0,0,0
1,0.791667,0,1,0,0,0,1,0.622642,0.365297,1,...,0,0,0,1,0,1,0,0,0,2
2,0.791667,0,1,0,0,0,1,0.245283,0.235160,1,...,0,0,1,0,0,0,0,1,0,1
3,0.166667,0,1,0,0,1,0,0.339623,0.283105,1,...,1,0,0,0,0,1,0,0,0,0
4,0.250000,1,0,0,1,0,0,0.339623,0.178082,1,...,1,0,0,0,0,1,0,0,0,0


In [23]:
X = new_df.values[:,:-1].astype('float')
target = new_df.values[:,-1]

encoder = LabelEncoder()
y = encoder.fit_transform(target)

n_input = X.shape[1]

if np.unique(y).shape[0] <= 2:
  print('Binary classification')
  n_output = 1

else:
  print('Multiclass classification')
  y = to_categorical(y)
  n_output = y.shape[1]


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

Multiclass classification


NameError: name 'to_categorical' is not defined

In [ ]:
model = NeuralNetwork(hidden_neurons = hidden_neurons, n_input=n_input, n_output=n_output, lr=lr, output_activation=output_activation)
model.train(X_train, y_train, verbose=1, loss='binary_crossentropy', epochs=epochs)

#### Resposta

In [ ]:
model.predict(X_test,y_test)

## Parte III - Mudança de configurações do modelo

### 1)	Insira o conjunto de validação para o treinamento do modelo. Avalie o resultado obtido.

#### Rascunho

#### Resposta

### 2)	Modifique o tempo de treinamento (épocas) da Rede Neural. Escolha valores distintos (e.g. 1 e 1000 épocas) e avalie os resultados.

#### Rascunho

#### Resposta

### 3)	 Modifique a taxa de aprendizado da Rede Neural. Escolha valores distintos (e.g. 0,001 e 0,1) e avalie os resultados.

#### Rascunho

#### Resposta

### 4)	Modifique a quantidade de neurônios na camada escondida da Rede Neural. Escolha valores distintos (e.g. 2 e 70 neurônios) e avalie os resultados.

#### Rascunho

#### Resposta

## Parte IV - Análise dos resultados

### 1)	Faça novos testes para avaliar o desempenho da Rede Neural no problema designado usando a técnica K-Fold (com K = 10). Comente o resultado obtido.

Dica: não é necessário utilizar a ferramenta KFold do scikit-learn para o desenvolvimento deste item. Uma alternativa <b> simples </b> é fazer um *for loop*, selecionando os índices em bloco. 

#### Rascunho

#### Resposta

### 2)	Faça análises e novas implementações que você julgue importante para o seu trabalho. Não esqueça de explicar a motivação da análise realizada. 

#### Rascunho

#### Resposta